In [1]:
import warnings

from docutils.nodes import classifier

warnings.filterwarnings("ignore")

from fss_funcs import fss
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
import numpy as np
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
from sklearn.tree import DecisionTreeClassifier
import wittgenstein as lw
from sklearn.model_selection import train_test_split
import time
from bayes_optimizable_mlpc import BayesOptimizableMLPC
from io import StringIO

rng = np.random.RandomState(42)

In [2]:
df = pd.read_csv("csv/outlier_filtered.csv")

response_var = 'Diabetes_012'
features = list(df.columns)
features.remove(response_var)

print(features, response_var)
# Pretty-print using tabulate
df.head(1)

['BMI_q_normal', 'MentHlth', 'PhysHlth_q_uniform', 'GenHlth_q_uniform', 'Age_q_uniform', 'Education_coxbox', 'Income_q_uniform', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex'] Diabetes_012


,BMI_q_normal,MentHlth,PhysHlth_q_uniform,GenHlth_q_uniform,Age_q_uniform,Education_coxbox,Income_q_uniform,Diabetes_012,HighBP,HighChol,...,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,DiffWalk,Sex
0,1.60221,1.998592,0.891892,1.0,0.581582,-1.109347,0.117117,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Features kept in multivariate feature selection
- K=08 -> [0, 2, 3, 4, 6, 7, 8, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19]

Features kept in univariate feature selection
- K=08 -> [0, 3, 4, 6, 7, 8, 12, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19]


In [3]:
"""
Features kept in multivariate feature selection
- K=08 -> [0, 2, 3, 4, 6, 7, 8, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19]

Features kept in univariate feature selection
- K=08 -> [0, 3, 4, 6, 7, 8, 12, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19]

Wrapper
K=08 -> [0, 3, 4, 6, 7, 8, 12, 16]
K=10 -> [0, 3, 4, 6, 7, 8, 9, 12, 16, 20]
K=12 -> [0, 3, 4, 6, 7, 8, 9, 11, 12, 16, 17, 19]
K=14 -> [0, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19]
K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19]
"""
fea_subsets = {
    'multivariate':{8:[0, 2, 3, 4, 6, 7, 8, 19],
               10:[0, 2, 3, 4, 5, 6, 7, 8, 12, 19],
               12:[0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19],
               14:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19],
               16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19]},
    'univariate':{8:[0, 3, 4, 6, 7, 8, 12, 19],
                  10:[0, 2, 3, 4, 5, 6, 7, 8, 12, 19],
                  12:[0, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 19],
                  14:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19],
                  16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19]},
    'wrapper':{8:[0, 3, 4, 6, 7, 8, 12, 16],
               10:[0, 3, 4, 6, 7, 8, 9, 12, 16, 20],
               12:[0, 3, 4, 6, 7, 8, 9, 11, 12, 16, 17, 19],
               14:[0, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19],
               16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 1]},
}

models = {
    'K-nearest neighbors' : BayesSearchCV(KNeighborsClassifier(),{
            'n_neighbors': Integer(3, 20),
            'weights': Categorical(['uniform', 'distance']),
            'metric': Categorical(['euclidean', 'manhattan', 'minkowski'])},
            n_iter=32,
            random_state=rng,
            n_jobs=-1,
            n_points=2,
        ),
    'Artificial neural networks': BayesSearchCV(
            BayesOptimizableMLPC(max_iter=1000,warm_start=True, random_state=rng, solver='adam'),
            search_spaces={
                'hidden_layer_sizes': Categorical(["32", "64", "32, 32", "64, 32", "64, 64", "32, 64, 32"]),  # Different layer structures
                'activation': Categorical(['identity', 'logistic', 'tanh', 'relu']),
                'alpha': Real(1e-4, 1e-2, prior='log-uniform'),  # L2 regularization term
                'batch_size': Integer(256, 2056),  # Size of minibatches for stochastic optimizers
            },
            random_state=rng,
            scoring='f1_macro',
            n_jobs=-1,
            n_points=4, 
            n_iter=20
        ),
'Support vector machines':  BayesSearchCV(
            SVC(random_state=rng,cache_size=1000),
            {
                'C': Real(1e-2, 1e2, prior='uniform'),
                'kernel': Categorical(['linear', 'rbf']),
                'gamma': Categorical(['scale', 'auto'])
            },
            random_state=rng,
            n_jobs=-1,
            n_points=4,
            scoring='f1_macro',
            cv=2,
            n_iter=2,
        ),
'Classification trees': BayesSearchCV(
            DecisionTreeClassifier(class_weight='balanced', random_state=rng),
            {
                'criterion': Categorical(['gini', 'entropy', 'log_loss']),
                'splitter': Categorical(['best', 'random']),
                'max_depth': Integer(5, 50),
                'min_samples_split': Real(1e-6, 0.1),
                'min_samples_leaf': Real(1e-6, 0.1),
                'class_weight': Categorical(['balanced']),
                'ccp_alpha': Real(0, 1),
            },
            random_state=rng,
            scoring='f1_macro',
            n_jobs=-1,
            n_points=4,
            n_iter=100
       )
}

binary_models = {
        "Rule Induction": BayesSearchCV(
            lw.RIPPER(random_state=rng),
            search_spaces={
                'k': Integer(1, 10),                          # Number of optimization rounds
                'prune_size': Real(0.1, 0.5),                 # Proportion of training set for pruning
                'dl_allowance': Integer(16, 128),             # Description length allowance for early stopping
                'n_discretize_bins': Integer(5, 20),          # Discretization bins
                'max_rules': Integer(1, 20),                  # Max number of rules
                'min_rule_samples': Integer(1, 10),           # Minimum samples per rule before halting rule growth
                'min_ruleset_samples': Integer(1, 20),        # Minimum samples in the entire ruleset
                'max_rule_conds': Integer(1, 10),             # Max number of conditions per rule
                'max_total_conds': Integer(10, 50),           # Max number of total conditions in ruleset
            },
            random_state=rng,
            scoring='f1_macro',
            n_jobs=-1,
            n_points=2,
        ),
}

# RIPPER PARAMS
"""
k : int, default=2
    Number of RIPPERk optimization iterations.
prune_size : float, default=.33
    Proportion of training set to be used for pruning.
dl_allowance : int, default=64
    Terminate Ruleset grow phase early if a Ruleset description length is encountered
    that is more than this amount above the lowest description length so far encountered.
n_discretize_bins : int, default=10
    Fit apparent numeric attributes into a maximum of n_discretize_bins discrete bins, inclusive on upper part of range. Pass None to disable auto-discretization.
random_state : int, default=None
    Random seed for repeatable results.

Limits for early-stopping. Intended for enhancing model interpretability and limiting training time on noisy datasets. Not specifically intended for use as a hyperparameter, since pruning already occurs during training, though it is certainly possible that tuning could improve model performance.
max_rules : int, default=None
    Maximum number of rules.
min_rule_samples : int, defulat=None
    Minimum number of covered samples per rule before halting rule growth.
min_ruleset_samples : int, default=None
    Minimum number of samples per rule before halting ruleset growth.
max_rule_conds : int, default=None
    Maximum number of conds per rule.
max_total_conds : int, default=None
    Maximum number of total conds in entire ruleset.
"""
pass

In [8]:
X_total, y_total = df[features], df[response_var]

# Define your desired fraction
fraction = 1

if fraction == 1:
    X_fraction, y_fraction = X_total, y_total
else:
    # Stratified sampling using train_test_split with only one split (fraction is the test_size here)
    X_fraction, _, y_fraction, _ = train_test_split(
        X_total, y_total,
        test_size=1-fraction,
        random_state=rng,
        stratify=y_total
    )

X_train, X_test, y_train, y_test = train_test_split(X_fraction, y_fraction,
                                                    train_size=0.9,
                                                    random_state=rng,
                                                    stratify=y_fraction)
print(X_train.shape, y_train.shape)

(228281, 21) (228281,)


In [ ]:
# Start tracking total runtime
def run_models(elapse=0, skip=["K-nearest neighbors", "Classification trees"]):
    start_total = time.time()
    
    y = y_train
    it = 0
    
    for model in models.keys():
        if model not in skip:
            print(f"Skipping {model}")
            continue 
            
        for fss_strategy in fea_subsets.keys():
            if fss_strategy in ['multivariate', 'univariate']:
                continue 
                
            for k in fea_subsets[fss_strategy].keys():
                if it < elapse:
                    print(f"Skipping it={it} with {model};{fss_strategy};{k};")
                    it += 1
                    continue 
                    
                # Start tracking time for this iteration
                start_time = time.time()
                
                # Define the feature subset and input data
                fea_subset = np.array(features)[fea_subsets[fss_strategy][k]]
                X = X_train[fea_subset]
                
                # Fit the model
                bayes_search = models[model]
                print(f"\rFitting {model}...                   ", end="")
                bayes_search.fit(X, y)
                print(f"\rFitting {model} completed            ", end="")
                # Get best parameters and score
                best_params = bayes_search.best_params_
                score = bayes_search.score(X_test[fea_subset], y_test)
                
                # Calculate elapsed time for this iteration
                elapsed_time = time.time() - start_time
                
                # Print results along with time taken
                print(f"\r{it};{model};{fss_strategy};{k};{best_params};{score:01.04f};{elapsed_time:06.02f}")
                it += 1
    
    # Calculate and print total runtime
    total_time = time.time() - start_total
    print(f"Total runtime: {total_time:.2f} seconds")

run_models(0)

Fitting K-nearest neighbors...                   

In [39]:
output = """
it; model; fss strategy; num fea; best parameters; score; elapsed time 
0;K-nearest neighbors;multivariate;8;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.8432879952690716;175.27
1;K-nearest neighbors;multivariate;10;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.8435639660950128;325.13
2;K-nearest neighbors;multivariate;12;OrderedDict({'metric': 'minkowski', 'n_neighbors': 15, 'weights': 'uniform'});0.8434456928838951;376.78
3;K-nearest neighbors;multivariate;14;OrderedDict({'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'});0.8434456928838951;628.67
4;K-nearest neighbors;multivariate;16;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.841986989946777;1286.11
5;K-nearest neighbors;univariate;8;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.8441553321506012;169.13
6;K-nearest neighbors;univariate;10;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.8435639660950128;303.49
7;K-nearest neighbors;univariate;12;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.8430514488468361;417.85
8;K-nearest neighbors;univariate;14;OrderedDict({'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'});0.8434456928838951;562.77
9;K-nearest neighbors;univariate;16;OrderedDict({'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'});0.8432879952690716;1303.31
10;Classification trees;multivariate;8;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.0551451310059547, 'min_samples_split': 0.1, 'splitter': 'random'});0.4217;049.93
11;Classification trees;multivariate;10;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.05781319163758467, 'min_samples_split': 0.08305687894819609, 'splitter': 'best'});0.4138;065.82
12;Classification trees;multivariate;12;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1e-06, 'min_samples_split': 0.045587443653603926, 'splitter': 'random'});0.4191;043.55
13;Classification trees;multivariate;14;OrderedDict({'ccp_alpha': 0.010679273044110017, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 8, 'min_samples_leaf': 0.1, 'min_samples_split': 0.006038931162415052, 'splitter': 'random'});0.4021;054.14
14;Classification trees;multivariate;16;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 50, 'min_samples_leaf': 1e-06, 'min_samples_split': 0.1, 'splitter': 'best'});0.4069;052.86
15;Classification trees;univariate;8;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 47, 'min_samples_leaf': 0.05915872872692148, 'min_samples_split': 0.1, 'splitter': 'random'});0.4081;045.71
16;Classification trees;univariate;10;OrderedDict({'ccp_alpha': 0.0008041567470032194, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 44, 'min_samples_leaf': 0.014046579680346154, 'min_samples_split': 0.028138888173554914, 'splitter': 'random'});0.4151;042.24
17;Classification trees;univariate;12;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.05464509991436246, 'min_samples_split': 1e-06, 'splitter': 'best'});0.4175;042.57
18;Classification trees;univariate;14;OrderedDict({'ccp_alpha': 0.0004095218216206843, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 13, 'min_samples_leaf': 0.03518870088484663, 'min_samples_split': 0.012278844580237428, 'splitter': 'random'});0.4141;053.75
19;Classification trees;univariate;16;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 5, 'min_samples_leaf': 1e-06, 'min_samples_split': 1e-06, 'splitter': 'random'});0.4230;070.15
0;Artificial neural networks;multivariate;8;OrderedDict({'activation': 'relu', 'alpha': 0.00034140471672701174, 'batch_size': 1666, 'hidden_layer_sizes': '64, 64'});0.4177;062.86
1;Artificial neural networks;multivariate;10;OrderedDict({'activation': 'relu', 'alpha': 0.003419694880629917, 'batch_size': 1669, 'hidden_layer_sizes': '64, 64'});0.4267;089.01
2;Artificial neural networks;multivariate;12;OrderedDict({'activation': 'identity', 'alpha': 0.008980741502394047, 'batch_size': 530, 'hidden_layer_sizes': '32, 32'});0.4093;033.12
3;Artificial neural networks;multivariate;14;OrderedDict({'activation': 'tanh', 'alpha': 0.00036249930490987373, 'batch_size': 1405, 'hidden_layer_sizes': '64, 32'});0.4504;124.56
4;Artificial neural networks;multivariate;16;OrderedDict({'activation': 'relu', 'alpha': 0.0002043075311149019, 'batch_size': 733, 'hidden_layer_sizes': '64'});0.4255;071.89
5;Artificial neural networks;univariate;8;OrderedDict({'activation': 'identity', 'alpha': 0.009090263265822638, 'batch_size': 256, 'hidden_layer_sizes': '64'});0.4168;039.34
6;Artificial neural networks;univariate;10;OrderedDict({'activation': 'identity', 'alpha': 0.0001, 'batch_size': 290, 'hidden_layer_sizes': '32, 32'});0.4031;065.41
7;Artificial neural networks;univariate;12;OrderedDict({'activation': 'relu', 'alpha': 0.0022471872460717193, 'batch_size': 1730, 'hidden_layer_sizes': '32, 32'});0.4480;089.53
8;Artificial neural networks;univariate;14;OrderedDict({'activation': 'relu', 'alpha': 0.0001, 'batch_size': 573, 'hidden_layer_sizes': '32, 32'});0.3982;137.80
9;Artificial neural networks;univariate;16;OrderedDict({'activation': 'relu', 'alpha': 0.01, 'batch_size': 256, 'hidden_layer_sizes': '64, 32'});0.3772;148.99
10;Artificial neural networks;wrapper;8;OrderedDict({'activation': 'identity', 'alpha': 0.0033704456334576983, 'batch_size': 644, 'hidden_layer_sizes': '64, 64'});0.4514;066.76
11;Artificial neural networks;wrapper;10;OrderedDict({'activation': 'relu', 'alpha': 0.0006407130905485844, 'batch_size': 562, 'hidden_layer_sizes': '64, 64'});0.4619;094.61
12;Artificial neural networks;wrapper;12;OrderedDict({'activation': 'identity', 'alpha': 0.0022509069396752495, 'batch_size': 1503, 'hidden_layer_sizes': '64, 64'});0.4290;053.14
13;Artificial neural networks;wrapper;14;OrderedDict({'activation': 'tanh', 'alpha': 0.0021227646072116114, 'batch_size': 462, 'hidden_layer_sizes': '32, 64, 32'});0.4584;186.02
14;Artificial neural networks;wrapper;16;OrderedDict({'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 415, 'hidden_layer_sizes': '64, 64'});0.4373;175.08
15;Support vector machines;multivariate;8;OrderedDict({'C': 96.18398405840975, 'gamma': 'scale', 'kernel': 'rbf'});0.4102;002.41
16;Support vector machines;multivariate;10;OrderedDict({'C': 82.90546877885846, 'gamma': 'scale', 'kernel': 'rbf'});0.4186;001.98
17;Support vector machines;multivariate;12;OrderedDict({'C': 73.65234245085956, 'gamma': 'scale', 'kernel': 'linear'});0.3048;164.02
18;Support vector machines;multivariate;14;OrderedDict({'C': 83.0457315447555, 'gamma': 'scale', 'kernel': 'rbf'});0.3956;052.42
19;Support vector machines;multivariate;16;OrderedDict({'C': 56.782865442341816, 'gamma': 'auto', 'kernel': 'rbf'});0.3916;001.54
20;Support vector machines;univariate;8;OrderedDict({'C': 60.43936696071759, 'gamma': 'scale', 'kernel': 'rbf'});0.4168;003.04
21;Support vector machines;univariate;10;OrderedDict({'C': 49.2794067550937, 'gamma': 'auto', 'kernel': 'rbf'});0.3908;022.68
22;Support vector machines;univariate;12;OrderedDict({'C': 50.63243829994258, 'gamma': 'scale', 'kernel': 'rbf'});0.3985;001.57
23;Support vector machines;univariate;14;OrderedDict({'C': 58.08196546177063, 'gamma': 'auto', 'kernel': 'linear'});0.3048;173.85
24;Support vector machines;univariate;16;OrderedDict({'C': 53.57606702721132, 'gamma': 'auto', 'kernel': 'rbf'});0.3732;079.02
25;Support vector machines;wrapper;8;OrderedDict({'C': 68.42622378245696, 'gamma': 'scale', 'kernel': 'rbf'});0.4108;002.45
26;Support vector machines;wrapper;10;OrderedDict({'C': 5.53453009201796, 'gamma': 'auto', 'kernel': 'rbf'});0.3139;001.59
27;Support vector machines;wrapper;12;OrderedDict({'C': 60.9607704374378, 'gamma': 'auto', 'kernel': 'rbf'});0.3546;002.12
28;Support vector machines;wrapper;14;OrderedDict({'C': 34.02024633416418, 'gamma': 'scale', 'kernel': 'rbf'});0.3933;007.43
29;Support vector machines;wrapper;16;OrderedDict({'C': 27.90304484687713, 'gamma': 'auto', 'kernel': 'rbf'});0.3950;003.95
"""

stats = pd.read_csv(StringIO(output), delimiter=";")
stats.drop('it')
stats

,it,model,fss strategy,num fea,best parameters,score,elapsed time
0,0,K-nearest neighbors,multivariate,8,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843288,175.27
1,1,K-nearest neighbors,multivariate,10,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843564,325.13
2,2,K-nearest neighbors,multivariate,12,"OrderedDict({'metric': 'minkowski', 'n_neighbo...",0.843446,376.78
3,3,K-nearest neighbors,multivariate,14,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843446,628.67
4,4,K-nearest neighbors,multivariate,16,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.841987,1286.11
5,5,K-nearest neighbors,univariate,8,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.844155,169.13
6,6,K-nearest neighbors,univariate,10,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843564,303.49
7,7,K-nearest neighbors,univariate,12,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843051,417.85
8,8,K-nearest neighbors,univariate,14,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843446,562.77
9,9,K-nearest neighbors,univariate,16,"OrderedDict({'metric': 'manhattan', 'n_neighbo...",0.843288,1303.31
